In [4]:
import os

from dotenv import load_dotenv
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_chroma import Chroma

from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# Load environment variables from .env
load_dotenv()

True

In [5]:
# Define the persistent directory
current_dir = os.getcwd()
persistent_directory = os.path.join(current_dir, "db", "chroma_db_with_metadata")

# Define the embedding model
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# Load the existing vector store with the embedding function
db = Chroma(persist_directory=persistent_directory, embedding_function=embeddings)

In [6]:
# Create a retriever for querying the vector store
# `search_type` specifies the type of search (e.g., similarity)
# `search_kwargs` contains additional arguments for the search (e.g., number of results to return)
retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3},
)
# Create a ChatOpenAI model
llm = ChatOpenAI(model="gpt-4o-mini")


In [7]:
# Contextualizar la pregunta
# Este prompt de sistema ayuda a la IA a entender que debe reformular la pregunta
# en función del historial de chat para convertirla en una pregunta autónoma
contextualize_q_system_prompt = (
    "Dado un historial de chat y la última pregunta del usuario "
    "que podría hacer referencia al contexto en el historial, "
    "formula una pregunta autónoma que pueda entenderse "
    "sin el historial de chat. NO respondas a la pregunta, solo "
    "reformula si es necesario; si no, devuélvela tal cual."
)


In [8]:
# Create a prompt template for contextualizing questions
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

# Create a history-aware retriever
# This uses the LLM to help reformulate the question based on chat history
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

In [9]:
# Responder a la pregunta
# Este prompt de sistema ayuda a la IA a entender que debe proporcionar respuestas concisas
# basadas en el contexto recuperado e indica qué hacer si la respuesta es desconocida
qa_system_prompt = (
    "Eres un asistente para tareas de preguntas y respuestas. Usa "
    "las siguientes piezas de contexto recuperado para responder a la "
    "pregunta. Si no sabes la respuesta, solo di que no la sabes. "
    "Usa un máximo de tres oraciones y mantén la respuesta "
    "concisa."
    "\n\n"
    "{context}"
)


In [10]:
# Create a prompt template for answering questions
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)


In [11]:
# Create a chain to combine documents for question answering
# `create_stuff_documents_chain` feeds all retrieved context into the LLM
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

# Create a retrieval chain that combines the history-aware retriever and the question answering chain
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [12]:
# Function to simulate a continual chat
def continual_chat():
    print("Start chatting with the AI! Type 'exit' to end the conversation.")
    chat_history = []  # Collect chat history here (a sequence of messages)
    while True:
        query = input("You: ")
        if query.lower() == "exit":
            break
        # Process the user's query through the retrieval chain
        result = rag_chain.invoke({"input": query, "chat_history": chat_history})
        # Display the AI's response
        print(f"AI: {result['answer']}")
        # Update the chat history
        chat_history.append(HumanMessage(content=query))
        chat_history.append(SystemMessage(content=result["answer"]))


# Main function to start the continual chat
if __name__ == "__main__":
    continual_chat()

Start chatting with the AI! Type 'exit' to end the conversation.
AI: Homero es un poeta épico de la antigua Grecia, tradicionalmente considerado el autor de las obras clásicas "La Ilíada" y "La Odisea". Su existencia histórica es objeto de debate, y algunos estudiosos sugieren que pudo haber sido un individuo o una recopilación de fragmentos de varios poetas. Su obra ha tenido un profundo impacto en la literatura y la cultura occidental.
AI: Algunos personajes importantes en "La Odisea" incluyen a Ulises (el protagonista y rey de Ítaca), Penélope (su fiel esposa), Telémaco (su hijo), Atenea (la diosa que lo ayuda), Poseidón (el dios del mar que le causa problemas), y Circe y Calypso (las diosas que lo retienen en sus islas). También son relevantes los pretendientes que asedian a Penélope en su ausencia y Eumeo, el porquero leal a Ulises. Estos personajes contribuyen a la exploración de temas como la lealtad, la astucia y el deseo de regresar a casa.
AI: No tengo una lista específica de